# Exploratory Data Analysis

This notebook is used to conduct exploratory data analysis on the "station_lines", "systems", "tracks" and "track_lines" tables of the "city-lines" dataset with the goal of uncovering data quality issues.

## Setup

### Import required packages

In [29]:
from pathlib import Path
import mysql.connector as connector
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'geopandas'

### Set credentials & establish connection with the database

In [2]:
credentials = {
    "username": "root",
    "password": ""
}

# Establish a connection with the DBMS
conn = connector.connect(user = credentials["username"],
                         passwd = credentials["password"],
                         host = "localhost",
                         database = "city-lines")

# Check if the connection is there
sql_query = """
SHOW tables;
"""
df = pd.read_sql(sql_query, conn)
df

,Tables_in_city-lines
0,cities
1,lines
2,station_lines
3,stations
4,systems
5,track_lines
6,tracks


### Load data

In [3]:
tables = {}
for table_name in ["station_lines", "systems", "tracks", "track_lines"]:
    sql_query = "SELECT * FROM {}".format(table_name)
    tables[table_name] = pd.read_sql(sql_query, conn)

## Analyze - 'station_lines' table

### Whole table

In [4]:
df = tables["station_lines"]

# Take a look at the head of the data
print(df.head(5))
print("\n")
# Generate a metadata report
print(df.info())

   id  station_id  line_id  city_id           created_at           updated_at
0   1        7694      629      114  2017-11-21 00:00:00  2017-11-21 00:00:00
1   2        6003      528       29  2017-11-21 00:00:00  2017-11-21 00:00:00
2   3        7732      570       74  2017-11-21 00:00:00  2017-11-21 00:00:00
3   4        7695      629      114  2017-11-21 00:00:00  2017-11-21 00:00:00
4   5        7726      570       74  2017-11-21 00:00:00  2017-11-21 00:00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16242 entries, 0 to 16241
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          16242 non-null  int64 
 1   station_id  16242 non-null  int64 
 2   line_id     16242 non-null  int64 
 3   city_id     16242 non-null  int64 
 4   created_at  16242 non-null  object
 5   updated_at  16242 non-null  object
dtypes: int64(4), object(2)
memory usage: 761.5+ KB
None


#### Table purpose and explanation
This table seems to be a 'relationship relation' table that lays out which stations lie on which lines. Inversely, it can also be used to represent the lines that pass from a station. It can also be linked to individual cities.

#### Summary

The "station_lines" table consists of six columns and 16242 rows. None of the rows have any null values, which is a good indicator. The first four columns (id, station_id, line_id and city_id) have the NumPy data type 'int64'. This makes sense as these are id's and id's are usually discrete numbers. This also means that id's are numeric only - no letters or markings at all. The last two columns (created_at and updated_at) show up as NumPy 'object' data type. This can mean anything.

#### Potential problems

- As the name of 'created_at' and 'updated_at' column suggests, these columns  were supposed to be 'time and date' data types. They did not show up in Pandas as such. This might be a problem while analyzing the data.


### 'station_id' column

In [5]:
target_col = df["station_id"]

# Number of unique values, what they are and how many times they appear
print(target_col.nunique()) # 15468 unique values
print("\n")
print(target_col.value_counts())

15468


11422    7
11258    6
10940    6
11049    6
11048    6
        ..
2308     1
2347     1
2309     1
2336     1
16558    1
Name: station_id, Length: 15468, dtype: int64


In [6]:
# What about the inverse, how many stations appear 7 times, 6 times ... 1 times etc.
result = (pd.DataFrame(target_col.value_counts())
          .rename({"station_id": "count"}, axis = 1))
result["station_id"] = result.index
result = (result
          .reset_index(drop = True)
          .groupby("count")
          .agg("count"))

print(result)

       station_id
count            
1           14851
2             502
3              91
4              13
5               5
6               5
7               1


In [7]:
# min and max of the column
print(target_col.min(), target_col.max())

1 16558


#### Column purpose and explanation
The "station_id" column seems to be a foreign key that links to "station" table.

#### Summary

There are 15468 unique stations in total. Most stations appear one time. However, some appear up to seven times.

#### Potential problems

No potential problems found.

### 'line_id' column

In [8]:
target_col = df["line_id"]

# Number of unique values, what they are and how many times they appear
print(target_col.nunique()) # 1020 unique values
print("\n")
print(target_col.value_counts())

1020


658     95
639     90
612     87
171     86
938     78
        ..
1213     1
1214     1
106      1
740      1
554      1
Name: line_id, Length: 1020, dtype: int64


In [9]:
# What about the inverse, how many stations appear 7 times, 6 times ... 1 times etc.
result = (pd.DataFrame(target_col.value_counts())
          .rename({"line_id": "count"}, axis = 1))
result["line_id"] = result.index
result = (result
          .reset_index(drop = True)
          .groupby("count")
          .agg("count"))

print(result)

       line_id
count         
1           34
2           89
3           58
4           48
5           33
...        ...
78           1
86           1
87           1
90           1
95           1

[69 rows x 1 columns]


In [10]:
# min and max of the column
print(target_col.min(), target_col.max())

5 1614


#### Column purpose and explanation

The 'line_id' column seems to be a foreign key that links to the "line" table.

#### Summary

There are 1020 unique lines in total, far less than stations as expected. The minimum and the maximum of the dataset also seems correct. The number of times that a line appears in the dataset has a more diverse distribution when compared to stations.

#### Potential problems

No potential problems found.

### 'city_id' column

In [11]:
target_col = df["city_id"]

# min and max of the column
print(target_col.min(), target_col.max())
print("\n")
# Number of unique values, what they are and how many times they appear
print(target_col.nunique()) # 81 unique values
print("\n")
print(target_col.value_counts())

1 331


81


114    3335
91     1291
206    1224
69      912
95      897
       ... 
283       4
199       3
211       2
45        1
72        1
Name: city_id, Length: 81, dtype: int64


In [12]:
# What about the inverse, how many stations appear n times, n - 1 times etc...
result = (pd.DataFrame(target_col.value_counts())
          .rename({"city_id": "count"}, axis = 1))
result["city_id"] = result.index
result = (result
          .reset_index(drop = True)
          .groupby("count")
          .agg("count"))

print(result)

       city_id
count         
1            2
2            1
3            1
4            1
5            1
...        ...
897          1
912          1
1224         1
1291         1
3335         1

[73 rows x 1 columns]


#### Column purpose and explanation

The 'city_id' column seems to be a foreign key that points to the 'cities' table.

#### Summary

The minimum id and the maximum id seems to be between logical bounds. There are 81 unique city id's in total. 

#### Potential problems

- There are 81 unique cities in total, which seems to be a problem.

### 'created_at' column

In [13]:
target_col = pd.to_datetime(df["created_at"]) # We need this as datetime

# min and max of the column
print(target_col.min(), target_col.max())
print("\n")

2017-11-21 00:00:00 2019-03-11 21:42:48.700022




#### Column purpose and explanation

The exact purpose of this column is a bit of a puzzle. It might be signalling the time that this entry was first created.

#### Summary

The min and the max values seem to be within logical bounds. Doesn't seem to be much useful for analysis purposes.

#### Potential problems

- This column needs to be of TIMESTAMP data type. It is currently text only.

### 'updated_at' column

In [14]:
target_col = pd.to_datetime(df["updated_at"]) # We need this as datetime

# min and max of the column
print(target_col.min(), target_col.max())
print("\n")

2017-11-21 00:00:00 2019-03-11 21:42:48.700022




#### Column purpose and explanation

The exact purpose of this column is a bit of a puzzle. It might be signalling the time that this entry was edited.

#### Summary

The min and the max values seem to be within logical bounds. Doesn't seem to be much useful for analysis purposes.

#### Potential problems

This column needs to be of TIMESTAMP data type. It is currently text only.

## Analyze - 'systems' table

### Whole table

In [15]:
df = tables["systems"]

# Take a look at the head of the data
print(df.head(5))
print("\n")
# Generate a metadata report
print(df.info())

   id  city_id  name
0   1        5  None
1   2        6  None
2   3        7  None
3   4        8  None
4   5        9  None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       488 non-null    int64 
 1   city_id  488 non-null    int64 
 2   name     287 non-null    object
dtypes: int64(2), object(1)
memory usage: 11.6+ KB
None


#### Table purpose and explanation

This table serves as a table that represents the "systems" entity. The systems entity has only one non-key attribute, and that is the 'name' attribute. The other two entities are id's. The 'id' column is the primary key of this table and the 'city_id' column is a foreign key that links to the 'cities' table.

#### Summary

The table has 3 columns and 488 rows. The first two columns ('id' and 'city_id') do not have any null values as they are keys. The 'name' column has nearly half of its values as NULL. The id columns are of NumPy data type int64 and the 'name' column is a NumPy object. This makes sense as the NumPy 'object' also describes text.

#### Potential problems


### 'id' column

In [22]:
target_col = df["id"]

# Check the min and max values
print(target_col.min(), target_col.max())
print("\n")

# Look at how many unique values there are and how many times they appear
print(target_col.nunique())
print("\n")
print(target_col.value_counts())

1 638


488


1      1
348    1
346    1
345    1
343    1
      ..
161    1
160    1
159    1
158    1
638    1
Name: id, Length: 488, dtype: int64


#### Column purpose and explanation

This column serves as the primary key of the 'systems' table.

#### Summary

The min and the max values seem to be within logical bounds. As expected from an id column, there are no null values and there are as many unique values as there are id's.

#### Potential problems

No potential problems found.

### 'city_id' column

In [24]:
target_col = df["city_id"]

# Check the min and max values
print(target_col.min(), target_col.max())
print("\n")

# Look at how many unique values there are and how many times they appear
print(target_col.nunique())
print("\n")
print(target_col.value_counts())
print("\n")

# What about the inverse, how many stations appear n times, n - 1 times etc...
result = (pd.DataFrame(target_col.value_counts())
          .rename({"city_id": "count"}, axis = 1))
result["city_id"] = result.index
result = (result
          .reset_index(drop = True)
          .groupby("count")
          .agg("count"))

print(result)

1 331


287


114    49
91     26
206    14
95      9
139     8
       ..
34      1
127     1
128     1
129     1
323     1
Name: city_id, Length: 287, dtype: int64


       city_id
count         
1          230
2           28
3           13
4            3
5            3
6            5
8            1
9            1
14           1
26           1
49           1


#### Column purpose and explanation

This column serves as a foreign key that links to the 'cities' table.

#### Summary

The min and the max values seem to be within logical bounds. As expected from an id column, there are no null values. Most city id's appear only once. However, some city id's appear more than once (up to 49 times). This might indicate that some cities possess a wider range of transportation systems.

#### Potential problems

No potential problems found.

### 'name' column

In [28]:
target_col = df["name"]

# Look at how many unique values there are and how many times they appear
print(target_col.nunique()) 
print("\n")
print(target_col.value_counts())
print("\n")

# What about the inverse, how many stations appear n times, n - 1 times etc...
result = (pd.DataFrame(target_col.value_counts())
          .rename({"name": "count"}, axis = 1))
result["city_id"] = result.index
result = (result
          .reset_index(drop = True)
          .groupby("count")
          .agg("count"))
print(result)

235


Tramway                                16
Metro                                  13
Tram                                    9
Métro                                   3
Funiculaire                             3
                                       ..
Tokyo Monorail                          1
Macrobús                                1
Tokyo Waterfront Area Rapid Transit     1
Comboios de Portugal                    1
Ómnibus                                 1
Name: name, Length: 235, dtype: int64


       city_id
count         
1          219
2            9
3            4
9            1
13           1
16           1


#### Column purpose and explanation

This column is the only non-key attribute of the 'systems' table. It gives information about what a transportation system is called in the local language.

#### Summary

There appears to be some universal names (like "Tramway" and "Metro") that appear more than once. Other than that, there aren't any exactly equal names. This table is not suitable to be used in the analysis of transportation system diversity because the systems with the same technical classification might be named or spelled differently in different languages.

#### Potential problems

No potential problems found.